# Trans-Effects with t tests

Here we look for genes NOT in the event discussed that are affected by the arm-level event. We will find these effects by performing a series of t-tests comparing the proteomic values of the patients with the event against those without the event.

## Setup (Install necessary packages)

We will start by importing necessary packages and collecting all of the proteomics data we will need to run the tests. The cancer types analyzed should have been determined in 01_event_basic_info where we determine which types of cancer seem to have the event we are looking at.

In [1]:
import cnvutils
import cptac
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy import stats
import seaborn as sns

In [2]:
# Load parameters
gen_params = cnvutils.load_params(os.path.join("..", "..", "..", "data", "gen_params.json"))
PANCAN = gen_params["PANCAN"]

chr_params = cnvutils.load_params(os.path.join("..", "..", "data", "chr_params.json"))
CHROMOSOME = chr_params["CHROMOSOME"]

arm_params = cnvutils.load_params(os.path.join("..", "data", "arm_params.json"))
ARM = arm_params["ARM"]
CANCER_TYPES = arm_params["ARM_CANCER_TYPES"]
EVENT_START = arm_params["START"]
EVENT_END = arm_params["END"]

# Set cis or trans
CIS_OR_TRANS = "trans"

# Get data tables
data_types = ["proteomics"]
tables = cnvutils.load_tables(CANCER_TYPES, data_types, pancan=PANCAN)
proteomics = tables["proteomics"]

EVENT_COLUMN = "event"

/home/caleb/anaconda3/envs/dev/lib/python3.9/site-packages/gtfparse/read_gtf.py:151: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version.


  result = parse_gtf(
/home/caleb/anaconda3/envs/dev/lib/python3.9/site-packages/gtfparse/read_gtf.py:151: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  result = parse_gtf(


## Select the proteins we're interested in

If we're looking at cis effects, we select proteins within the event. If we're looking at trans effects, we select proteins outside of the event.

Note that the cnvutils.get_event_genes function uses Ensembl gene IDs for the Database_ID column, while the proteomics dataframes that have a Database_ID column use RefSeq protein IDs. So, when we're selecting the genes we want, we ignore the Database_ID column if it is present, and just use gene names.

In [3]:
selected_genes = cnvutils.\
get_event_genes(
    chrm=CHROMOSOME,
    event_start=EVENT_START,
    event_end=EVENT_END,
    cis_or_trans=CIS_OR_TRANS
)["Name"].\
drop_duplicates(keep="first")

for cancer_type in proteomics.keys():
    df = proteomics[cancer_type].transpose()
    
    if df.index.nlevels == 1:
        df = df[df.index.isin(selected_genes)]
    else:
        df = df[df.index.isin(selected_genes, level="Name")]

    proteomics[cancer_type] = df

## Append Event Data

We now append the data from the event table that should have been created in a previous notebook.

In [4]:
has_event = dict()
for cancer_type in proteomics.keys():
    df = proteomics[cancer_type]
    df = df.transpose()
    event = pd.read_csv(os.path.join(
        "..", 
        "data", 
        f"chr{CHROMOSOME}_{cancer_type}_has_event_{'harmonized' if PANCAN else 'AWG'}.tsv"
    ), sep='\t', index_col=0)
    event.index.rename('Name')
    df = df.join(event)
    df = df.dropna(subset=[EVENT_COLUMN])
    has_event[cancer_type] = df[EVENT_COLUMN]
    proteomics[cancer_type] = df

/home/caleb/anaconda3/envs/dev/lib/python3.9/site-packages/pandas/core/frame.py:9130: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left,1 on the right)
  return merge(


## Run T-Tests

In [5]:
results_df = None
for cancer_type in proteomics.keys():
    prot_df = proteomics[cancer_type]
    results = cptac.utils.wrap_ttest(
        df=prot_df, 
        label_column=EVENT_COLUMN,
        correction_method="fdr_bh",
        return_all=True
    )
    results.set_index('Comparison', inplace=True)
    if isinstance(results.index[0], tuple):
        results[['Name', f'{cancer_type}_Database_ID']] = pd.DataFrame(
            results.index.values.tolist(),
            index=results.index
        )
        results.set_index(['Name', f'{cancer_type}_Database_ID'], inplace=True)
    else:
        results.index.name='Name'
    results.rename(columns={'P_Value': f'{cancer_type}_pvalue'}, inplace=True)
    if results_df is None:
        results_df = results
    else:
        results_df = results_df.join(results)

## Append Difference Data

We want to collect one more piece of information about the data. We want to find the difference between the averages of the two groups. This will tell us if there is a positive or negative coorelation between the event and the proteomic data and can give us some indication of the strength of the coorelation. 

In [6]:
def get_diff(col, event):
    has_event = col[event]
    invert_list = [not x for x in event]
    no_event = col[invert_list]
    event_avg = has_event.mean()
    no_event_avg = no_event.mean()
    return event_avg - no_event_avg

In [7]:
diff_df = None
for cancer_type in proteomics.keys():
    df = proteomics[cancer_type]
    df = df.drop(EVENT_COLUMN, axis=1)
    results = df.apply(lambda x: get_diff(x, has_event[cancer_type]))
    df = pd.DataFrame(results)
    if isinstance(df.index[0], tuple):
        df[['Name', f'{cancer_type}_Database_ID']] = pd.DataFrame(df.index.values.tolist(), index=df.index)
        df.set_index(['Name', f'{cancer_type}_Database_ID'], inplace=True)
    else:
        df.index.name='Name'
    df.rename(columns={0: f'{cancer_type}_diff'}, inplace=True)
    if diff_df is None:
        diff_df = df
    else:
        diff_df = diff_df.join(df)

## Join the tables, reformat, and save

We now join the difference table and the results table together, then reformat them nicely. We also save the table to a tsv for use in future analyses. 

In [8]:
results_df = results_df.join(diff_df)

In [9]:
results_df = results_df.\
reset_index(drop=False).\
rename(columns={"Name": "protein"}).\
set_index("protein")

long_results = pd.DataFrame()

for cancer_type in CANCER_TYPES:
    cancer_df = results_df.\
    loc[:, results_df.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)

    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)

    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)

    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]

    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [10]:
save_path = os.path.join(
    "..", 
    "data", 
    f"chr{CHROMOSOME}{ARM}_{CIS_OR_TRANS}_00_ttest_{'harmonized' if PANCAN else 'AWG'}.tsv"
)
long_results.to_csv(save_path, sep='\t', index=False)